In [ ]:
pip install -q yahoo_fin > /dev/null

In [ ]:
pip install --upgrade yfinance > /dev/null

In [3]:
mkdir -p data results log

In [4]:
from google.colab import drive
drive.mount('/content/drive')
# ! find /content/drive/MyDrive/colab/ -type f -mtime +180 -delete -print


Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd

In [6]:
def show_prediction(name, suffix):
  df = pd.read_csv(f'results/{name}_{suffix}_predictions.csv')
  df['predicted_return'] = ((df['predicted_max']+df['predicted_min']+df['predicted_close'])/3 - df['current_price'])/df['current_price']
  return df.sort_values('predicted_return', ascending=False, key=abs)


In [7]:
def run_command_with_check(command):
  """Runs a command, prints stderr, and raises an exception if it fails."""
  process = subprocess.run(command, shell=True, capture_output=True, text=True, check=False)

  if process.returncode != 0:
    print("stderr:", process.stderr)  # Print stderr only on error
    raise subprocess.CalledProcessError(process.returncode, process.args, output=process.stdout, stderr=process.stderr)
  else:
    # Print or process stdout if needed
    #print("stdout:", process.stdout)
    pass

  return process.returncode


In [8]:
import os
import subprocess
def run_model_nn_cmd(prefix, version, do_train=False, window_size=None, future_days=None, hourly=False, copy_tickers=True):
    # copy drive/MyDrive/colab/results/nnreturn.py to local folder
    tickers = f"{prefix}tickers.csv"
    run_command_with_check(f"cp  /content/drive/MyDrive/colab/nnreturn.py .")
    if copy_tickers:
      run_command_with_check(f"cp  /content/drive/MyDrive/colab/tickers/{tickers} .")
    if not do_train:
      run_command_with_check(f"cp  /content/drive/MyDrive/colab/models/{prefix}_{version}_*.pth .")
      # Run nnreturn.py in results folder and throw exception if error
      today = pd.to_datetime('today').strftime('%m%d')
      run_command_with_check(f"rename 's/{version}/{today}/' *.pth")
      cmd_arg = ""
      if window_size:
        cmd_arg += f" --window-size {window_size}"
      if future_days:
        cmd_arg += f" --future-days {future_days}"
      if hourly:
        cmd_arg += " --hourly"
      run_command_with_check(f"python3 nnreturn.py --tickers-file {prefix}tickers.csv --mode analyze {cmd_arg}")
      run_command_with_check(f"cp {prefix}*predictions.csv results")
      # Get today's date

      return show_prediction(prefix, today)
    else:
      raise Exception("Not implemented")


In [ ]:
run_model_nn_cmd('etf', '0730').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
4,DXYZ,27.08,2026-02-24,2026-03-24,26.29,31.85,29.35,0.076933
31,XME,119.60,2026-02-24,2026-03-24,116.09,134.02,128.96,0.056494
23,XLE,54.95,2026-02-24,2026-03-24,53.55,57.71,59.50,0.035851
21,XLB,53.40,2026-02-24,2026-03-24,49.07,54.36,51.06,-0.035643
0,ARKF,37.37,2026-02-24,2026-03-24,37.12,39.80,39.07,0.034609
8,IXC,50.83,2026-02-24,2026-03-24,49.16,53.39,54.57,0.030363
2,ARKW,121.81,2026-02-24,2026-03-24,119.58,132.00,120.09,0.017076
14,JETS,28.94,2026-02-24,2026-03-24,28.74,30.37,29.10,0.016010
19,URA,54.62,2026-02-24,2026-03-24,50.35,59.25,51.86,-0.014647
1,ARKK,71.95,2026-02-24,2026-03-24,69.39,76.10,67.24,-0.014454


In [ ]:
# 0716 is hourly 20
run_model_nn('idx', '0716', window_size=20, future_days=8, hourly=True).head(10)

In [ ]:
import runml.pipeline
from runml import pipeline,findata
#from importlib import reload
# reload(runml.pipeline)

In [ ]:
# Skip some days from consecutive window to lessen test /train overlap
# findata.G_NON_OVERLAP= 0

Run following if like to save results in drive

In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    # Simplify following using apply
    def calculate_alloc(row):
        gain_low = min(row['Gain_l'], 0)
        gain_high = max(row['Gain_h'], 0)
        gain = min(row['Gain'], gain_high)
        average_gain = (gain_low+gain_high + 2*gain)/4
        if (average_gain == 0):
          return 0
        avg_accuracy = (row['Accu']+row['Accu_l']+row['Accu_h'])/3
        alloc =  avg_accuracy/stop_loss - (1-avg_accuracy)*profit_factor/abs(average_gain)
        return round(max(alloc, 0), 2)
    df['Alloc'] = df.apply(calculate_alloc, axis=1)
    return df

pd.options.display.max_columns = None

In [ ]:
import os
# findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  # Save datafrme to prefix-version-prediction.csv
  df.to_csv(f"results/{prefix}-{version}-prediction.csv")
  return df



In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax' # Change to minmax
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'BYRN', 'CELH', 'CFLT', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ELF', 'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HCP', 'HIMS', 'HOOD', 'HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JAKK', 'JKS', 'KD', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR', 'MRVL', 'MXL','MTCH', 'MU',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PAR', 'PANW',  'PAYX', 'PD', 'PI', 'PINS',
           'RBLX', 'RKLB', 'RMBS', 'RUM', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'SOUN', 'TEM', 'TEAM', 'TDOC', 'TMDX', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '2a', tickers, True)

In [ ]:
addAllocHL(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'DXYZ',
            'EEM', 'FDN', 'FPX',
            'ICLN', 'IGV', 'IJK', 'IJR', 'IPO',
            'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'JXI', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'URA', 'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '1b', tickers1, True)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

In [ ]:
df1.to_csv('results/etf_12b_predictions.csv')

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax'
tickers3 = [ 'AIRI', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS',  'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CHGG', 'CLOV', 'CRCT', 'CXM',
            'DLO', 'DM', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MITK', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PLUG', 'PTON', 'PUBM',
            'REAL', 'REI', 'RIG', 'RVYL',
            'SABR', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'VTEX', 'VSAT', 'YEXT']
df3 = run_model('vols', '12b', tickers3, False)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'KO', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
df4 = run_model('mcap', '1b', tickers4, True)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CX', 'DBOEY', 'DNNGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FDRVF', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']

df5 = run_model('gstock', '2a', tickers5, False)

In [ ]:
addAllocHL(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.G_SCALER = 'standard'
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'KWT', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '12a', tickers6, False)

In [ ]:
addAllocHL(df6, 0.07, 1)
df6.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'standard'
tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE', 'ASC',
            'BAH', 'BAX', 'BKR', 'BLK', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IFF', 'IP', 'IPI', 'IVZ',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW', 'ZTS']
df7 = run_model('val', '1a', tickers7, False)

In [ ]:
addAllocHL(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers8 = [ 'DBC', 'FXB', 'FXE', 'FXF', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '12a', tickers8, True)

In [ ]:
addAllocHL(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

In [ ]:
dfp = run_model_nn_cmd('pennystock', '1226')

In [ ]:
dfp[dfp['predicted_return'] > 0].head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
93,GDRX,2.43,2026-02-25,2026-03-25,2.34,3.00,2.90,0.130316
10,BLNK,0.70,2026-02-25,2026-03-25,0.61,1.01,0.75,0.128571
33,MAX,9.62,2026-02-25,2026-03-25,8.46,12.07,11.51,0.110187
107,UDMY,5.11,2026-02-25,2026-03-25,4.83,6.28,5.81,0.103718
7,AWRE,1.65,2026-02-25,2026-03-25,1.58,2.02,1.84,0.098990
54,SLE,3.49,2026-02-25,2026-03-25,3.20,4.97,3.33,0.098376
55,SMWB,2.63,2026-02-25,2026-03-25,2.53,3.08,3.00,0.091255
82,AEHR,40.91,2026-02-25,2026-03-25,38.02,49.64,45.95,0.088650
70,CYN,1.50,2026-02-25,2026-03-25,1.41,2.08,1.38,0.082222
31,LFMD,2.67,2026-02-25,2026-03-25,2.28,3.47,2.90,0.079900


In [ ]:
run_model_nn_cmd('bluechipstock', '0221').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
34,PYPL,45.73,2026-02-26,2026-03-26,46.40,51.09,48.33,0.062905
26,LLY,1009.32,2026-02-26,2026-03-26,1006.65,1108.92,1086.56,0.057521
28,MMC,181.10,2026-02-25,2026-03-25,181.95,202.04,187.07,0.051095
33,ORCL,149.36,2026-02-26,2026-03-26,129.25,156.33,141.76,-0.046286
30,NKE,64.39,2026-02-26,2026-03-26,61.88,70.73,69.46,0.046073
18,DIS,106.72,2026-02-26,2026-03-26,104.89,118.00,111.41,0.044165
38,TSLA,409.86,2026-02-26,2026-03-26,392.58,439.32,444.12,0.037769
2,ADBE,260.57,2026-02-26,2026-03-26,255.08,285.06,270.31,0.036766
36,SBUX,98.76,2026-02-26,2026-03-26,95.89,105.43,105.31,0.034933
32,NOW,109.03,2026-02-26,2026-03-26,105.79,119.79,112.81,0.034547


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '11b', tickers9, False)

In [ ]:
addAllocHL(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

In [ ]:
run_model_nn_cmd('country', '1112').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
23,VNM,18.03,2026-02-06,2026-03-06,16.98,18.18,17.72,-0.022370
18,INDA,53.28,2026-02-06,2026-03-06,50.26,53.86,52.47,-0.020333
20,KWT,37.25,2026-02-06,2026-03-06,35.10,38.37,36.35,-0.017271
10,EWM,29.57,2026-02-06,2026-03-06,27.96,31.30,30.87,0.016007
9,EWC,55.01,2026-02-06,2026-03-06,54.05,57.82,55.45,0.013876
11,EWP,56.46,2026-02-06,2026-03-06,55.25,59.98,56.38,0.013166
22,TUR,40.84,2026-02-06,2026-03-06,39.41,42.86,41.75,0.012243
19,KEMQ,25.92,2026-02-06,2026-03-06,24.58,26.43,25.80,-0.012217
5,EWA,28.23,2026-02-06,2026-03-06,27.37,29.54,28.81,0.012162
6,EWG,43.53,2026-02-06,2026-03-06,42.56,44.85,44.59,0.010797


In [9]:
run_model_nn_cmd('global', '1107').head(15)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
20,FQVLF,30.06,2026-02-27,2026-03-27,25.00,31.15,27.78,-0.069306
55,VALE,17.18,2026-02-27,2026-03-27,15.16,17.68,15.21,-0.067714
52,TKAMY,12.41,2026-02-27,2026-03-27,11.55,12.95,14.72,0.053452
11,DNNGY,7.92,2026-02-27,2026-03-27,7.34,8.74,8.95,0.053451
5,CAAP,28.50,2026-02-27,2026-03-27,25.47,28.66,27.05,-0.050526
60,TKC,6.74,2026-02-27,2026-03-27,6.03,6.96,6.23,-0.049456
14,ENGGY,9.00,2026-02-27,2026-03-27,8.88,9.89,9.50,0.047037
25,HMC,30.16,2026-02-27,2026-03-27,29.71,32.33,32.63,0.046309
39,OTGLY,17.00,2026-02-27,2026-03-27,14.82,17.46,16.43,-0.044902
19,FANUY,22.65,2026-02-27,2026-03-27,21.96,25.09,23.87,0.043709


In [ ]:
run_model_nn_cmd('valuestock', '0301').head(15)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
41,FLO,11.96,2026-02-06,2026-03-06,11.79,13.51,13.50,0.081382
18,CCJ,113.63,2026-02-06,2026-03-06,85.67,118.47,110.88,-0.075890
64,OSK,171.71,2026-02-06,2026-03-06,145.67,174.38,158.15,-0.071691
1,ADI,320.45,2026-02-06,2026-03-06,286.59,320.40,288.06,-0.068966
71,POWL,584.22,2026-02-06,2026-03-06,516.15,587.87,532.03,-0.066533
96,WSM,220.67,2026-02-06,2026-03-06,194.74,218.50,213.69,-0.052990
86,TGT,115.55,2026-02-06,2026-03-06,103.90,116.84,108.32,-0.050743
60,NFG,85.77,2026-02-06,2026-03-06,85.40,94.97,88.77,0.045976
58,MOD,215.72,2026-02-06,2026-03-06,208.78,245.35,222.00,0.044765
99,ZTS,127.42,2026-02-06,2026-03-06,128.98,132.04,136.74,0.040548


In [ ]:
run_model_nn_cmd('commodity', '0312').head(5)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
4,FXY,59.53,2026-02-23,2026-03-23,58.22,62.05,60.24,0.010751
15,WEAT,22.11,2026-02-23,2026-03-23,21.57,22.91,22.42,0.008593
11,TLT,89.91,2026-02-23,2026-03-23,87.58,93.04,91.05,0.007192
12,USO,81.17,2026-02-23,2026-03-23,79.38,83.88,81.97,0.007063
9,SOYB,23.65,2026-02-23,2026-03-23,23.02,24.34,23.88,0.004087


In [ ]:
run_model_nn_cmd('crypto', '0214').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
6,DOGE-USD,0.09,2026-02-23,2026-03-23,0.09,0.10,0.13,0.185185
17,ATOM-USD,2.14,2026-02-23,2026-03-23,2.19,2.83,2.26,0.133956
8,LTC-USD,51.92,2026-02-23,2026-03-23,51.99,58.28,63.79,0.117488
12,AVAX-USD,8.53,2026-02-23,2026-03-23,7.54,8.79,12.07,0.109809
5,DOT-USD,1.28,2026-02-23,2026-03-23,1.26,1.52,1.45,0.101563
3,ADA-USD,0.27,2026-02-23,2026-03-23,0.26,0.29,0.34,0.098765
4,XRP-USD,1.37,2026-02-23,2026-03-23,1.37,1.49,1.63,0.092457
7,FIL-USD,0.89,2026-02-23,2026-03-23,0.87,0.99,1.04,0.086142
2,SOL-USD,79.16,2026-02-23,2026-03-23,81.26,91.46,84.47,0.082996
13,LINK-USD,8.38,2026-02-23,2026-03-23,8.14,9.50,9.46,0.077963


In [ ]:
df = run_model_nn_cmd('techstock', '0312')
df[df['predicted_return']>0].head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
19,DOCU,43.31,2026-02-24,2026-03-24,41.22,57.77,47.63,0.128454
44,MNDY,75.37,2026-02-24,2026-03-24,74.21,86.95,89.53,0.108708
52,NTNX,37.96,2026-02-24,2026-03-24,37.41,45.44,42.37,0.099579
74,TTD,24.63,2026-02-24,2026-03-24,24.01,28.65,28.24,0.094871
69,TDOC,4.50,2026-02-24,2026-03-24,4.14,5.60,4.89,0.083704
60,PINS,17.02,2026-02-24,2026-03-24,16.49,19.10,19.30,0.075010
30,GTLB,25.18,2026-02-24,2026-03-24,24.57,28.83,27.19,0.066852
2,AI,9.81,2026-02-24,2026-03-24,9.39,11.09,10.87,0.065240
55,PAR,19.73,2026-02-24,2026-03-24,20.13,23.54,19.09,0.060314
67,SPOT,466.38,2026-02-24,2026-03-24,453.54,554.46,470.88,0.056992


In [17]:
run_model_nn_cmd('stock', '0227', window_size=40).head(20)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
36,UP,0.60,2026-02-27,2026-03-27,0.52,0.99,0.70,0.227778
69,DUOL,98.00,2026-02-27,2026-03-27,95.04,136.48,118.14,0.189320
320,INOD,44.43,2026-02-27,2026-03-27,42.01,58.87,52.90,0.153725
132,LFMD,2.65,2026-02-27,2026-03-27,2.50,3.07,3.35,0.122013
199,MSTR,128.49,2026-02-27,2026-03-27,123.86,174.15,133.09,0.118375
5,DOMO,3.66,2026-02-27,2026-03-27,3.52,4.54,4.11,0.108379
0,SE,109.13,2026-02-27,2026-03-27,103.90,142.48,115.00,0.103821
221,ASTS,78.68,2026-02-27,2026-03-27,63.30,83.68,65.33,-0.100534
194,GDYN,6.72,2026-02-27,2026-03-27,6.50,7.96,7.69,0.098710
274,QTWO,48.44,2026-02-27,2026-03-27,47.06,58.29,53.92,0.095995


In [ ]:
run_model_nn_cmd('buystocks', '0402', False, 500, 250).head(10)
# show_prediction('buystocks', '0508').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
342,OPEN,4.78,2026-02-04,2027-01-20,4.40,9.34,9.02,0.587169
46,EL,117.68,2026-02-04,2027-01-20,116.14,209.73,219.13,0.543734
156,INTC,47.65,2026-02-04,2027-01-20,45.20,88.05,86.21,0.535222
321,AEVA,13.06,2026-02-04,2027-01-20,11.22,23.51,24.11,0.501787
115,VOD,15.61,2026-02-04,2027-01-20,15.69,27.03,26.85,0.485586
93,CHGG,0.70,2026-02-04,2027-01-20,0.59,1.29,1.23,0.480952
228,SLDP,3.99,2026-02-04,2027-01-20,3.07,7.61,6.99,0.476190
219,PAGS,10.78,2026-02-04,2027-01-20,8.18,20.82,18.32,0.463203
324,ENPH,50.22,2026-02-04,2027-01-20,48.44,96.73,74.20,0.456060
348,REMX,84.97,2026-02-04,2027-01-20,82.66,162.47,123.48,0.446040


In [ ]:
run_model_nn_cmd('techstock', '0224', False, 500, 250).head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
76,RBLX,67.09,2026-02-24,2027-02-09,60.44,129.31,119.58,0.536891
98,UI,758.69,2026-02-24,2027-02-09,674.71,1480.07,1333.45,0.532567
77,RMBS,98.57,2026-02-24,2027-02-09,77.40,187.03,159.78,0.434547
85,SYNA,81.06,2026-02-24,2027-02-09,72.87,156.04,114.11,0.410560
42,HOOD,73.39,2026-02-24,2027-02-09,53.04,143.05,113.82,0.407594
96,U,18.29,2026-02-24,2027-02-09,13.21,33.19,30.43,0.400219
51,LYFT,13.53,2026-02-24,2027-02-09,10.40,24.61,21.32,0.387780
102,XYZ,51.00,2026-02-24,2027-02-09,46.32,89.32,76.53,0.386732
61,NET,163.81,2026-02-24,2027-02-09,117.50,297.29,258.36,0.369778
95,TWLO,113.85,2026-02-24,2027-02-09,94.80,213.00,156.70,0.359977


In [ ]:
run_model_nn_cmd('etf', '0225', False, 500, 250).head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
2,ARKW,127.36,2026-02-25,2027-02-10,104.27,233.10,213.17,0.440902
19,URA,55.91,2026-02-25,2027-02-10,45.55,111.66,84.43,0.440649
0,ARKF,39.11,2026-02-25,2027-02-10,31.04,67.24,57.83,0.330521
32,XME,120.68,2026-02-25,2027-02-10,95.07,206.56,167.50,0.295796
1,ARKK,73.94,2026-02-25,2027-02-10,60.91,115.11,106.08,0.271752
17,SMH,426.83,2026-02-25,2027-02-10,318.37,628.84,572.22,0.186600
10,IXP,120.99,2026-02-25,2027-02-10,105.64,155.11,153.09,0.140149
31,XLY,116.86,2026-02-25,2027-02-10,107.39,143.45,144.19,0.126790
23,XLC,116.18,2026-02-25,2027-02-10,100.89,147.56,141.63,0.119183
27,XLK,143.10,2026-02-25,2027-02-10,112.55,192.00,173.66,0.113930


In [ ]:
run_model_nn_cmd('bluechipstock', '0117', False, 500, 250).head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
5,AVGO,328.46,2026-01-29,2027-01-14,287.62,608.10,568.04,0.485478
32,ORCL,164.71,2026-01-29,2027-01-14,152.25,315.91,249.25,0.451865
21,INTC,48.06,2026-01-29,2027-01-14,43.10,83.98,73.79,0.393189
17,DIS,111.24,2026-01-29,2027-01-14,106.58,174.07,155.86,0.308013
6,BA,234.45,2026-01-29,2027-01-14,223.97,336.86,318.81,0.250643
22,IBM,304.70,2026-01-29,2027-01-14,292.43,439.38,406.63,0.245422
39,UNH,292.80,2026-01-29,2027-01-14,151.07,325.39,187.66,-0.243944
18,GOOGL,335.86,2026-01-29,2027-01-14,301.50,478.89,438.51,0.209730
13,CSCO,78.73,2026-01-29,2027-01-14,74.46,108.06,98.34,0.189127
30,NVDA,191.13,2026-01-29,2027-01-14,150.83,254.82,273.16,0.183854


In [ ]:
run_model_nn_cmd('pennystock', '0122', False, 500, 250).head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
109,AUID,1.63,2026-02-12,2026-03-12,1.48,2.54,1.86,0.202454
12,CHGG,0.57,2026-02-12,2026-03-12,0.54,0.94,0.54,0.181287
15,CXM,5.39,2026-02-12,2026-03-12,5.23,7.00,6.33,0.147805
64,ALAR,7.22,2026-02-12,2026-03-12,4.98,8.15,5.66,-0.132502
108,MRAM,11.00,2026-02-12,2026-03-12,10.61,13.87,12.87,0.131818
66,IREN,41.04,2026-02-12,2026-03-12,29.70,42.13,35.42,-0.128899
85,WKEY,6.80,2026-02-12,2026-03-12,5.93,9.05,8.04,0.128431
38,ONDS,8.97,2026-02-12,2026-03-12,6.43,10.12,7.13,-0.120030
40,OTLY,11.08,2026-02-12,2026-03-12,9.92,13.37,13.91,0.119134
107,UDMY,4.51,2026-02-12,2026-03-12,4.32,5.82,4.93,0.113821


In [ ]:
run_model_nn_cmd('global', '0226', False, 500, 250).head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
62,ASM,9.10,2026-02-26,2027-02-11,7.98,18.16,17.91,0.613553
43,RYCEF,18.50,2026-02-26,2027-02-11,17.42,36.29,35.73,0.611532
69,NVTS,9.51,2026-02-26,2027-02-11,7.57,18.56,17.82,0.540484
26,KNNGF,74.00,2026-02-25,2027-02-10,68.16,135.48,127.25,0.490495
47,SE,109.23,2026-02-26,2027-02-11,96.75,210.92,177.28,0.479905
61,SIFY,16.10,2026-02-26,2027-02-11,12.72,30.12,28.16,0.469979
45,SFTBY,13.05,2026-02-26,2027-02-11,9.14,24.40,23.71,0.462324
20,FQVLF,29.79,2026-02-26,2027-02-11,28.28,56.88,44.80,0.454179
3,BAK,3.73,2026-02-26,2027-02-11,3.60,6.33,5.94,0.418231
65,KEP,22.26,2026-02-26,2027-02-11,18.67,39.56,36.38,0.416742


In [ ]:
run_model_nn_cmd('valuestock', '0219', False, 500, 250).head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
86,NUTX,99.59,2026-02-20,2027-02-05,81.31,186.03,192.93,0.540550
113,STRL,433.26,2026-02-20,2027-02-05,315.79,859.64,826.69,0.540353
80,MP,55.18,2026-02-20,2027-02-05,50.21,103.51,97.98,0.520478
24,CCJ,118.41,2026-02-20,2027-02-05,103.98,225.22,203.20,0.498747
96,POWL,539.63,2026-02-20,2027-02-05,431.36,968.34,894.65,0.417237
61,GLW,137.42,2026-02-20,2027-02-05,112.34,238.97,214.61,0.372726
130,WDC,290.10,2026-02-20,2027-02-05,178.91,478.56,493.51,0.322509
30,CLS,296.71,2026-02-20,2027-02-05,219.38,513.79,441.41,0.319560
87,OLN,23.90,2026-02-20,2027-02-05,22.58,37.70,33.63,0.309763
23,CCI,88.12,2026-02-20,2027-02-05,86.67,137.41,118.60,0.296263


In [ ]:
run_model_nn_cmd('crypto', '0223', False, 500, 250).head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
16,AAVE-USD,114.99,2026-02-23,2027-02-08,42.99,154.25,48.05,-0.288953
17,MATIC-USD,0.22,2025-03-24,2026-03-09,0.17,0.41,0.26,0.272727
0,BTC-USD,64800.95,2026-02-23,2027-02-08,24188.92,82484.52,36648.62,-0.262757
3,ADA-USD,0.26,2026-02-23,2027-02-08,0.16,0.49,0.24,0.141026
13,LINK-USD,8.67,2026-02-22,2027-02-07,6.08,15.14,8.39,0.138408
7,FIL-USD,0.89,2026-02-23,2027-02-08,0.45,1.16,0.70,-0.134831
12,AVAX-USD,8.87,2026-02-22,2027-02-07,6.98,16.72,6.46,0.133408
6,DOGE-USD,0.09,2026-02-23,2027-02-08,0.05,0.13,0.06,-0.111111
8,LTC-USD,53.47,2026-02-22,2027-02-07,25.67,95.68,56.87,0.111028
5,DOT-USD,1.27,2026-02-23,2027-02-08,0.68,1.87,0.85,-0.107612


In [ ]:
import sys
sys.path.append('/content')
!unzip drive/MyDrive/colab/stock_prediction_r.zip
import stock_prediction.stock_prediction.stock_predictor_lib
from stock_prediction.stock_prediction.stock_predictor_lib import run_model_nn, show_prediction, StockPredictorLib, Config
colab_path = 'content/drive/MyDrive/colab'

In [ ]:
techcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
colab_path = 'content/drive/MyDrive/colab'
techcfg.model_path = f'/{colab_path}/models/transformer/techstock_1220'
techdf = run_model_nn(f'/{colab_path}/tickers/techstocktickers.csv', cfg=techcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/techstock_1220_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/techstock_1220_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/techstock_1220_close_model.pth'}


In [ ]:
techdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
48,MU,375.85,2026-02-10,2026-03-10,227.85,382.23,223.76,-0.260485
44,MNDY,76.30,2026-02-10,2026-03-10,72.68,108.91,99.80,0.229314
11,COHR,230.74,2026-02-10,2026-03-10,172.44,237.92,138.73,-0.206770
63,RMBS,107.14,2026-02-10,2026-03-10,70.81,110.35,80.81,-0.184960
68,TEAM,96.41,2026-02-10,2026-03-10,92.85,126.74,122.82,0.183868
61,PUBM,6.84,2026-02-10,2026-03-10,6.75,9.32,8.00,0.173002
34,HUBS,239.07,2026-02-10,2026-03-10,215.79,318.56,304.50,0.169602
77,UI,717.46,2026-02-10,2026-03-10,486.38,727.43,588.56,-0.162615
49,NET,180.16,2026-02-10,2026-03-10,135.93,185.46,133.18,-0.158951
76,U,29.05,2026-02-10,2026-03-10,21.34,30.65,21.81,-0.153184


In [ ]:
techrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
techrcfg.model_path = f'/{colab_path}/models/rolling/techstock_0109'
techrdf = run_model_nn(f'/{colab_path}/tickers/techstocktickers.csv', cfg=techrcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/techstock_0109_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/techstock_0109_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/techstock_0109_close_model.pth'}


In [ ]:
techrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
16,DDOG,104.39,2026-02-24,2026-03-24,101.30,142.78,116.89,0.152633
33,HOOD,72.36,2026-02-24,2026-03-24,64.66,89.09,92.82,0.135849
18,DOCS,23.88,2026-02-24,2026-03-24,22.60,28.59,29.66,0.128559
13,CVNA,330.17,2026-02-24,2026-03-24,313.04,400.59,387.20,0.111377
60,PINS,16.97,2026-02-24,2026-03-24,13.61,18.06,13.75,-0.107837
20,DT,34.46,2026-02-24,2026-03-24,31.74,43.60,38.20,0.098278
5,APP,389.27,2026-02-24,2026-03-24,293.08,437.03,333.79,-0.088979
2,AI,9.89,2026-02-24,2026-03-24,9.49,12.69,10.10,0.087968
63,RMBS,100.12,2026-02-24,2026-03-24,94.40,119.09,111.96,0.083533
55,PAR,19.77,2026-02-24,2026-03-24,16.56,21.85,16.41,-0.075704


In [ ]:
defetfcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
defetfcfg.model_path = f'/{colab_path}/models/rolling/etf_1221'
etfdf = run_model_nn(f'/{colab_path}/tickers/etftickers.csv', cfg=defetfcfg)
etfdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/etf_1221_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/etf_1221_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/etf_1221_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
22,XLC,116.91,2026-01-27,2026-02-24,114.02,121.17,127.73,0.034756
21,XLB,49.99,2026-01-27,2026-02-24,48.59,53.51,52.07,0.028006
0,ARKF,46.05,2026-01-27,2026-02-24,45.11,49.12,47.49,0.025841
31,XME,128.38,2026-01-27,2026-02-24,125.85,138.44,128.85,0.020772
19,URA,57.85,2026-01-27,2026-02-24,56.05,63.46,57.59,0.020455
3,DIA,490.07,2026-01-27,2026-02-24,479.53,513.22,507.28,0.020283
28,XLU,43.41,2026-01-27,2026-02-24,40.92,44.09,42.89,-0.017891
16,QQQ,631.13,2026-01-27,2026-02-24,615.90,651.98,656.09,0.016151
8,IXC,46.34,2026-01-27,2026-02-24,45.23,48.58,47.37,0.015537
15,JXI,81.94,2026-01-27,2026-02-24,77.62,83.46,81.52,-0.013099


In [ ]:
tvolcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
tvolcfg.model_path = f'/{colab_path}/models/transformer/pennystock_1226'
tvoldf = run_model_nn(f'/{colab_path}/tickers/pennystocktickers.csv', cfg=tvolcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/pennystock_1226_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/pennystock_1226_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/pennystock_1226_close_model.pth'}


In [ ]:
tvoldf[tvoldf['predicted_return']> 0].head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
2,AMSSY,5.74,2026-02-24,2026-03-24,5.63,8.61,15.48,0.725900
54,SLE,3.49,2026-02-25,2026-03-25,3.42,6.82,3.86,0.346705
91,PETS,2.79,2026-02-25,2026-03-25,2.75,4.56,3.10,0.243728
89,GPRO,0.84,2026-02-25,2026-03-25,0.74,1.16,1.10,0.190476
109,AUID,1.50,2026-02-25,2026-03-25,1.25,1.96,2.12,0.184444
6,ATOM,5.74,2026-02-25,2026-03-25,4.16,8.66,7.40,0.174216
90,DNUT,2.99,2026-02-25,2026-03-25,2.80,3.95,3.22,0.111483
31,LFMD,2.69,2026-02-25,2026-03-25,2.61,3.49,2.81,0.104089
69,BNED,8.32,2026-02-25,2026-03-25,4.69,12.04,10.64,0.096554
28,INVZ,0.85,2026-02-25,2026-03-25,0.72,1.19,0.87,0.090196


In [ ]:
tvolrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
tvolrcfg.model_path = f'/{colab_path}/models/rolling/pennystock_0114'
tvolrdf = run_model_nn(f'/{colab_path}/tickers/pennystocktickers.csv', cfg=tvolrcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/pennystock_0114_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/pennystock_0114_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/pennystock_0114_close_model.pth'}


In [ ]:
tvolrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
2,AMSSY,5.19,2026-02-11,2026-03-11,4.30,7.20,7.22,0.202312
54,SLE,4.35,2026-02-12,2026-03-12,3.63,6.57,5.29,0.186973
106,DVLT,0.74,2026-02-12,2026-03-12,0.61,1.24,0.78,0.184685
8,AXTI,25.86,2026-02-12,2026-03-12,22.28,35.43,33.81,0.179685
33,MAX,7.48,2026-02-12,2026-03-12,7.03,10.13,8.97,0.164439
87,SERV,9.44,2026-02-12,2026-03-12,7.30,10.05,6.77,-0.148305
109,AUID,1.64,2026-02-12,2026-03-12,1.13,2.34,2.10,0.132114
74,EOSE,11.34,2026-02-12,2026-03-12,10.61,15.89,11.95,0.130218
76,TSSI,8.15,2026-02-12,2026-03-12,7.20,10.22,10.11,0.125971
40,OTLY,11.04,2026-02-12,2026-03-12,9.68,11.74,15.73,0.121679


In [ ]:
bctcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
bctcfg.model_path = f'/{colab_path}/models/transformer/bluechipstock_1010'
bctdf = run_model_nn(f'/{colab_path}/tickers/bluechipstocktickers.csv', cfg=bctcfg)
bctdf.head(10)

In [ ]:
bcrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
bcrcfg.model_path = f'/{colab_path}/models/rolling/bluechipstock_0117'
bcrdf = run_model_nn(f'/{colab_path}/tickers/bluechipstocktickers.csv', cfg=bcrcfg)
bcrdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_0117_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_0117_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_0117_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
14,CSCO,78.43,2026-02-26,2026-03-26,71.19,80.25,69.55,-0.060776
25,KO,80.25,2026-02-26,2026-03-26,78.57,90.78,85.56,0.058816
15,DE,620.78,2026-02-26,2026-03-26,607.38,703.49,655.81,0.056026
8,BKNG,4268.91,2026-02-26,2026-03-26,3543.37,4446.64,4099.32,-0.056017
5,ANET,130.25,2026-02-26,2026-03-26,124.23,150.03,137.54,0.053871
23,IBM,244.44,2026-02-26,2026-03-26,227.92,267.40,274.73,0.050087
33,ORCL,148.46,2026-02-26,2026-03-26,138.79,166.40,159.95,0.044367
9,BRK-B,499.40,2026-02-26,2026-03-26,484.76,553.69,521.03,0.040902
7,BA,228.42,2026-02-26,2026-03-26,223.73,252.98,236.35,0.040569
39,TXN,213.21,2026-02-26,2026-03-26,205.29,232.98,226.08,0.038647


In [ ]:
buystkcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
buystkcfg.model_path = f'/{colab_path}/models/transformer/buystocks_0123'
buystkcfg.model.window_size = 500
buystkcfg.model.future_days = 250
buystkdf = run_model_nn(f'/{colab_path}/tickers/buystockstickers.csv', cfg=buystkcfg)

In [ ]:
buystkdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
28,BNED,9.43,2026-01-22,2027-01-07,8.50,10.09,248.54,8.442559
94,CHGG,0.79,2026-01-22,2027-01-07,0.78,11.47,5.72,6.582278
269,CRNC,11.49,2026-01-22,2027-01-07,11.16,124.59,99.59,5.827386
10,SFIX,5.22,2026-01-22,2027-01-07,5.04,50.66,44.06,5.370370
86,RIVN,16.32,2026-01-22,2027-01-07,13.62,118.22,99.29,3.720792
18,INVZ,1.13,2026-01-22,2027-01-07,1.10,7.17,7.46,3.640118
55,LCID,11.47,2026-01-22,2027-01-07,11.07,74.41,57.85,3.165359
35,UP,0.83,2026-01-22,2027-01-07,0.61,6.72,1.30,2.465863
17,OUST,27.24,2026-01-22,2027-01-07,11.33,132.54,112.54,2.137665
144,PTON,5.81,2026-01-22,2027-01-07,5.82,26.19,18.97,1.924842


In [ ]:
buystkrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
buystkrcfg.model_path = f'/{colab_path}/models/rolling/buystocks_1213'
buystkrcfg.model.window_size = 500
buystkrcfg.model.future_days = 250
buystkrdf = run_model_nn(f'/{colab_path}/tickers/buystockstickers.csv', cfg=buystkrcfg)

In [ ]:
buystkrdf.head(10)

In [ ]:
stkcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
stkcfg.model_path = f'/{colab_path}/models/transformer/stock_0105'
stkdf = run_model_nn(f'/{colab_path}/tickers/stocktickers.csv', cfg=stkcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/stock_0105_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/stock_0105_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/stock_0105_close_model.pth'}


In [ ]:
stkdf[stkdf['predicted_return'] > 0].head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
371,SQNS,3.62,2026-02-09,2026-03-09,3.26,5.10,5.10,0.239411
29,BNED,8.70,2026-02-09,2026-03-09,8.07,12.64,10.85,0.209195
251,ENVX,6.57,2026-02-09,2026-03-09,5.84,7.33,10.46,0.198884
36,UP,0.69,2026-02-09,2026-03-09,0.58,1.12,0.78,0.198068
187,PUBM,6.55,2026-02-09,2026-03-09,6.25,9.23,7.95,0.192366
97,CHGG,0.75,2026-02-09,2026-03-09,0.68,1.06,0.84,0.146667
65,GLOB,58.39,2026-02-09,2026-03-09,52.20,72.62,74.67,0.138837
108,TWLO,115.44,2026-02-09,2026-03-09,108.31,126.86,153.15,0.121275
357,ETHA,15.80,2026-02-09,2026-03-09,14.78,17.91,19.88,0.109072
30,AI,11.60,2026-02-09,2026-03-09,10.99,15.29,12.13,0.103736


In [12]:
stkrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
stkrcfg.model_path = f'/{colab_path}/models/rolling/stock_0102'
stkrdf = run_model_nn(f'/{colab_path}/tickers/stocktickers.csv', cfg=stkrcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/stock_0102_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/stock_0102_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/stock_0102_close_model.pth'}


In [13]:
stkrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
113,APP,430.67,2026-02-27,2026-03-27,290.40,448.83,345.82,-0.160185
225,FSLY,18.25,2026-02-27,2026-03-27,17.32,26.10,19.78,0.154338
285,VERX,14.36,2026-02-27,2026-03-27,9.50,15.64,11.44,-0.150882
8,HIMS,15.01,2026-02-27,2026-03-27,14.05,18.32,18.91,0.138796
221,ASTS,79.49,2026-02-27,2026-03-27,57.65,85.21,64.36,-0.131044
63,WDAY,132.27,2026-02-27,2026-03-27,127.98,165.53,151.33,0.121040
56,LCID,9.98,2026-02-27,2026-03-27,7.78,15.39,10.36,0.119906
2,SOUN,8.77,2026-02-27,2026-03-27,6.74,9.35,7.17,-0.115926
31,CREX,3.41,2026-02-27,2026-03-27,2.55,3.63,2.90,-0.112414
376,CRCL,84.48,2026-02-27,2026-03-27,68.51,131.29,81.43,0.109651


In [ ]:
glbcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
glbcfg.model_path = f'/{colab_path}/models/transformer/global_0105'
glbdf = run_model_nn(f'/{colab_path}/tickers/globaltickers.csv', cfg=glbcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/global_0105_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/global_0105_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/global_0105_close_model.pth'}


In [ ]:
glbdf[glbdf['predicted_return']>0].head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
14,EDNMF,21.94,2026-02-13,2026-03-13,22.34,30.20,26.34,0.198420
60,ALAR,6.58,2026-02-26,2026-03-26,5.89,8.47,8.76,0.171226
43,PROSF,51.05,2026-02-25,2026-03-25,47.33,67.48,56.33,0.117467
52,TGLS,44.42,2026-02-26,2026-03-26,44.01,55.08,49.24,0.113087
11,DNNGY,8.07,2026-02-26,2026-03-26,7.58,10.42,8.75,0.104915
50,STLA,7.96,2026-02-26,2026-03-26,7.90,8.97,8.22,0.050670
67,HSAI,26.51,2026-02-26,2026-03-26,25.19,31.31,26.57,0.044512
40,OTGLY,17.43,2026-02-26,2026-03-26,15.83,19.18,19.35,0.039587
34,NHYDY,9.10,2026-02-26,2026-03-26,8.93,9.82,9.54,0.036264
33,LVMUY,129.66,2026-02-26,2026-03-26,129.77,133.38,139.89,0.036146


In [ ]:
glbrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
glbrcfg.model_path = f'/{colab_path}/models/rolling/global_1210'
glbrdf = run_model_nn(f'/{colab_path}/tickers/globaltickers.csv', cfg=glbrcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/global_1210_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/global_1210_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/global_1210_close_model.pth'}


In [ ]:
glbrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
14,EDNMF,20.60,2026-02-06,2026-03-06,20.18,30.31,29.91,0.300971
53,TKAMY,12.35,2026-02-13,2026-03-13,12.13,14.46,14.09,0.097976
23,GLOB,50.24,2026-02-13,2026-03-13,48.33,57.22,58.17,0.086253
5,CAAP,27.99,2026-02-13,2026-03-13,27.02,33.39,30.28,0.080029
6,COTY,2.56,2026-02-13,2026-03-13,2.53,2.96,2.71,0.067708
2,ADYEY,10.66,2026-02-13,2026-03-13,10.06,11.72,12.34,0.066917
62,SIFY,14.84,2026-02-13,2026-03-13,13.78,16.43,17.17,0.064241
27,KNNGF,75.70,2026-01-22,2026-02-19,60.03,78.81,74.17,-0.062043
36,NU,16.70,2026-02-13,2026-03-13,16.19,18.71,18.08,0.057485
24,HCMLY,18.17,2026-02-13,2026-03-13,17.04,20.69,19.75,0.054485


In [ ]:
ctcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
ctcfg.model_path = f'/{colab_path}/models/transformer/country_1211'
ctdf = run_model_nn(f'/{colab_path}/tickers/countrytickers.csv', cfg=ctcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/country_1211_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/country_1211_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/country_1211_close_model.pth'}


In [ ]:
ctdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
4,EPHE,26.73,2026-02-06,2026-03-06,25.15,26.83,24.98,-0.040279
24,XCEM,42.88,2026-02-06,2026-03-06,40.69,42.86,41.31,-0.029384
8,EWJ,89.40,2026-02-06,2026-03-06,84.34,89.46,87.43,-0.025988
19,KEMQ,25.92,2026-02-06,2026-03-06,25.85,27.15,26.75,0.025592
18,INDA,53.28,2026-02-06,2026-03-06,52.89,55.35,55.05,0.021584
21,THD,65.75,2026-02-06,2026-03-06,61.27,66.26,65.70,-0.020380
16,IDX,15.55,2026-02-06,2026-03-06,15.41,16.19,15.97,0.019721
11,EWP,56.46,2026-02-06,2026-03-06,53.17,57.02,55.97,-0.019011
0,ARGT,93.16,2026-02-06,2026-03-06,91.49,98.77,94.31,0.018212
20,KWT,37.25,2026-02-06,2026-03-06,37.13,37.71,35.01,-0.017002


In [ ]:
valtcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
valtcfg.model_path = f'/{colab_path}/models/transformer/valuestock_1018'
valtdf = run_model_nn(f'valuestocktickers.csv', cfg=valtcfg)


In [ ]:
valtdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
71,POWL,355.91,2025-11-12,2025-12-10,318.63,406.16,497.69,0.144934
16,CC,12.51,2025-11-12,2025-12-10,11.58,15.90,14.90,0.129230
7,ARE,53.86,2025-11-12,2025-12-10,52.99,59.89,66.73,0.111586
91,UAL,99.94,2025-11-12,2025-12-10,91.43,119.48,121.23,0.107798
14,BXC,57.66,2025-11-12,2025-12-10,55.51,69.67,63.39,0.090126
44,GLW,88.71,2025-11-12,2025-12-10,86.23,108.45,94.80,0.087739
74,RH,160.72,2025-11-12,2025-12-10,156.87,193.12,172.52,0.083686
37,F,13.45,2025-11-12,2025-12-10,12.72,13.75,10.53,-0.083024
58,MOD,147.99,2025-11-12,2025-12-10,142.12,179.25,154.26,0.071311
39,FICO,1778.19,2025-11-12,2025-12-10,1648.44,2256.72,1784.05,0.066480


In [ ]:
valrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
valrcfg.model_path = f'/{colab_path}/models/rolling/valuestock_0106'
valrdf = run_model_nn(f'/{colab_path}/tickers/valuestocktickers.csv', cfg=valrcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/valuestock_0106_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/valuestock_0106_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/valuestock_0106_close_model.pth'}


In [ ]:
valrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
4,ANF,97.66,2026-02-06,2026-03-06,88.33,114.06,114.75,0.082463
89,TWI,11.06,2026-02-06,2026-03-06,10.74,12.06,13.04,0.080169
63,OLN,24.83,2026-02-06,2026-03-06,21.66,25.23,22.68,-0.066049
82,STRL,401.29,2026-02-06,2026-03-06,342.84,411.94,371.96,-0.064068
83,SWK,89.39,2026-02-06,2026-03-06,81.00,91.61,78.89,-0.062162
67,PETS,2.88,2026-02-06,2026-03-06,2.76,3.21,3.13,0.053241
71,POWL,584.22,2026-02-06,2026-03-06,564.52,657.37,621.67,0.051864
16,CC,18.32,2026-02-06,2026-03-06,16.19,19.71,16.28,-0.050582
45,GWW,1197.65,2026-02-06,2026-03-06,1017.33,1236.10,1168.14,-0.047699
28,CSL,403.86,2026-02-06,2026-03-06,361.30,419.62,373.08,-0.047525


In [ ]:
commtcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
commtcfg.model_path = f'/{colab_path}/models/transformer/commodity_0109'
commtdf = run_model_nn(f'/{colab_path}/tickers/commoditytickers.csv', cfg=commtcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/commodity_0109_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/commodity_0109_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/commodity_0109_close_model.pth'}


In [ ]:
commtdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
8,SLV,78.92,2026-02-23,2026-03-23,62.06,77.86,63.79,-0.139593
3,FXF,114.15,2026-02-23,2026-03-23,112.61,124.85,118.66,0.039918
11,TLT,89.92,2026-02-23,2026-03-23,89.26,95.16,94.66,0.034549
5,GLD,478.79,2026-02-23,2026-03-23,450.76,475.00,470.16,-0.028161
4,FXY,59.52,2026-02-23,2026-03-23,58.93,61.36,62.39,0.023073
0,DBC,24.77,2026-02-23,2026-03-23,24.39,25.63,25.25,0.012919
15,WEAT,22.12,2026-02-23,2026-03-23,21.08,23.10,22.70,0.007836
12,USO,81.57,2026-02-23,2026-03-23,75.81,88.98,78.28,-0.006702
13,UUP,27.06,2026-02-23,2026-03-23,26.95,27.58,27.12,0.005790
6,ISHG,76.39,2026-02-23,2026-03-23,75.22,77.19,75.45,-0.005716


In [ ]:
cryptcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
cryptcfg.model_path = f'/{colab_path}/models/transformer/crypto_1021'
cryptdf = run_model_nn(f'/{colab_path}/tickers/cryptotickers.csv', cfg=cryptcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_close_model.pth'}


In [ ]:
cryptdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
15,XLM-USD,0.23,2025-11-21,2025-12-19,0.22,0.28,0.29,0.144928
6,DOGE-USD,0.14,2025-11-21,2025-12-19,0.14,0.16,0.18,0.142857
3,ADA-USD,0.41,2025-11-21,2025-12-19,0.39,0.48,0.53,0.138211
4,XRP-USD,1.96,2025-11-21,2025-12-19,1.93,2.34,2.39,0.132653
5,DOT-USD,2.37,2025-11-21,2025-12-19,2.25,3.10,2.64,0.123769
11,ICP-USD,4.36,2025-11-21,2025-12-19,3.64,4.84,5.74,0.087156
12,AVAX-USD,13.32,2025-11-21,2025-12-19,8.87,16.84,11.20,-0.076326
17,MATIC-USD,0.22,2025-03-24,2025-04-21,0.15,0.26,0.20,-0.075758
18,ATOM-USD,2.53,2025-11-21,2025-12-19,2.31,3.29,2.50,0.067194
16,AAVE-USD,157.80,2025-11-21,2025-12-19,152.88,185.84,162.92,0.059654


In [ ]:
cryprcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
cryprcfg.model_path = f'/{colab_path}/models/rolling/crypto_0109'
cryprdf = run_model_nn(f'/{colab_path}/tickers/cryptotickers.csv', cfg=cryprcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/crypto_0109_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/crypto_0109_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/crypto_0109_close_model.pth'}


In [ ]:
cryprdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
6,DOGE-USD,0.09,2026-02-23,2026-03-23,0.09,0.11,0.10,0.111111
1,ETH-USD,1886.96,2026-02-23,2026-03-23,1330.20,2174.07,1709.21,-0.079034
18,ATOM-USD,2.14,2026-02-23,2026-03-23,2.02,2.54,2.33,0.073209
2,SOL-USD,78.93,2026-02-23,2026-03-23,59.76,87.89,72.00,-0.072385
0,BTC-USD,65295.14,2026-02-23,2026-03-23,59270.92,81705.72,67312.41,0.063321
17,MATIC-USD,0.22,2025-03-24,2025-04-21,0.19,0.24,0.19,-0.060606
4,XRP-USD,1.37,2026-02-23,2026-03-23,1.08,1.60,1.24,-0.046229
11,ICP-USD,2.05,2026-02-23,2026-03-23,1.90,2.29,1.72,-0.039024
16,AAVE-USD,116.05,2026-02-23,2026-03-23,101.88,128.02,106.10,-0.034899
13,LINK-USD,8.41,2026-02-23,2026-03-23,7.21,10.21,8.39,0.022989


In [ ]:
etftcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
etftcfg.model_path = f'/{colab_path}/models/transformer/etf_0112'
etftdf = run_model_nn(f'/{colab_path}/tickers/etftickers.csv', cfg=etftcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/etf_0112_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/etf_0112_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/etf_0112_close_model.pth'}


In [ ]:
etftdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
31,XME,123.38,2026-02-11,2026-03-11,104.60,124.62,105.14,-0.096666
19,URA,53.65,2026-02-11,2026-03-11,44.68,54.80,47.64,-0.085927
23,XLE,54.99,2026-02-11,2026-03-11,48.13,55.63,48.13,-0.079287
7,IPO,43.96,2026-02-11,2026-03-11,37.34,46.65,40.99,-0.052320
12,IWO,338.79,2026-02-11,2026-03-11,311.52,339.83,312.51,-0.051664
8,IXC,50.64,2026-02-11,2026-03-11,45.65,51.14,48.24,-0.045353
27,XLP,88.52,2026-02-11,2026-03-11,81.04,90.41,83.08,-0.041535
10,IXP,122.49,2026-02-11,2026-03-11,120.59,132.12,129.64,0.040493
20,VNQ,93.29,2026-02-11,2026-03-11,87.51,95.13,86.39,-0.038732
21,XLB,53.56,2026-02-11,2026-03-11,47.63,53.66,53.70,-0.035412


In [ ]:
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
from google.auth import default


auth.authenticate_user()


creds, _ = default()
gc = gspread.authorize(creds)
# gc = gspread.authorize(GoogleCredentials.get_application_default())


In [ ]:
# Open your spreadsheet by name
spreadsheet = gc.open('Allocation')
# Or by URL: spreadsheet = gc.open_by_url('Your Spreadsheet URL')

# Select the worksheet (e.g., the first sheet)
worksheet = spreadsheet.worksheet("Buy")

# Get all values as a list of lists
data = worksheet.get_all_values()

# Convert to a Pandas DataFrame
sheetdf = pd.DataFrame(data[1:], columns=data[0]) # Assuming first row is header


In [ ]:
df = sheetdf[['Ticker']].copy()

# Apply the extraction function first, then filter based on the result
def extract_ticker(ticker_str):
    # Convert to string before splitting
    ticker_str = str(ticker_str)
    parts = ticker_str.split(':')
    if len(parts) > 1:
        return parts[1].strip() # Use strip to remove leading/trailing whitespace
    return ticker_str.strip() # Return the original string if no colon, also strip whitespace

df['Ticker'] = df['Ticker'].apply(extract_ticker)
# Remove BTCUSD and add BTC-USD, ETH-USD, ADA-USD, SOL-USD in Ticker
df = df[df['Ticker'] != 'BTCUSD']
df = pd.concat([df, pd.DataFrame({'Ticker': ['BTC-USD', 'ETH-USD', 'ADA-USD', 'SOL-USD']})], ignore_index=True)
df.to_csv(f'/{colab_path}/tickers/stocktickers.csv', index=False)
df.to_csv(f'/{colab_path}/tickers/buystockstickers.csv', index=False)

In [ ]:
# Only have Tickers column

run_model_nn('buystocks', '0402', False, 500, 250, copy_tickers=False).head(10)


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
333,QBTS,34.25,2025-10-08,2026-09-23,32.44,71.01,65.57,0.644964
96,CHGG,1.42,2025-10-08,2026-09-23,1.40,2.82,2.74,0.633803
97,SLP,16.47,2025-10-08,2026-09-23,16.09,32.07,31.69,0.616070
18,OUST,30.65,2025-10-08,2026-09-23,29.45,61.59,56.80,0.607830
9,HIMS,58.17,2025-10-08,2026-09-23,49.10,110.63,110.40,0.547934
252,LYFT,21.16,2025-10-08,2026-09-23,19.66,41.40,36.93,0.543636
6,DOMO,14.74,2025-10-08,2026-09-23,12.98,29.59,25.17,0.531886
326,ROOT,81.73,2025-10-08,2026-09-23,76.40,156.09,141.76,0.526367
122,LZ,9.97,2025-10-08,2026-09-23,9.63,19.32,16.65,0.524574
276,CRNC,11.96,2025-10-08,2026-09-23,11.62,21.43,21.43,0.518395


In [ ]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +180 -delete -print

sending incremental file list
buystocks_0709_predictions.csv

sent 18,897 bytes  received 35 bytes  37,864.00 bytes/sec
total size is 18,770  speedup is 0.99


In [ ]:

# update zip /tmp/stock_cache.zip from files in folder /tmp/stock_cache
!zip -r /tmp/stock_cache.zip /tmp/stock_cache
!cp /tmp/stock_cache.zip /content/drive/MyDrive/colab/

In [ ]:
# Copy zip file from /content/drive/MyDrive/colab/ to /tmp/stock_cache.zip
!cp /content/drive/MyDrive/colab/stock_cache.zip /tmp/stock_cache.zip
!unzip  /tmp/stock_cache.zip